In [39]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("./insurance.csv")

In [7]:
df.head(5)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
0,67,119.8,1.56,2.92,False,Jaipur,retired,High
1,36,101.1,1.83,34.28,False,Chennai,freelancer,Low
2,39,56.8,1.64,36.64,False,Indore,freelancer,Low
3,22,109.4,1.55,3.34,True,Mumbai,student,Medium
4,69,62.2,1.60,3.94,True,Indore,retired,High


In [9]:
df.sample(2)

,age,weight,height,income_lpa,smoker,city,occupation,insurance_premium_category
44,59,77.0,1.6,50.00,True,Lucknow,private_job,Medium
18,52,80.9,1.8,38.14,True,Kota,business_owner,High


In [10]:
df_feat = df.copy()

## Feature Engineering

In [11]:
#feature1
df_feat['bmi'] = df_feat['weight'] / (df_feat['height']**2)

In [12]:
#feature 2
def age_group(age):
    if age<25:
        return "young"
    elif age<45:
        return "adult"
    elif age<60:
        return "middle aged"
    return "senior"

In [13]:
df_feat['age_group'] = df_feat["age"].apply(age_group)

In [18]:
# feature 3
def lifestyle_risk(row):
    if row['smoker'] and row['bmi'] > 30:
        return "high"
    elif row["smoker"] and row['bmi']>27:
        return "medium"
    else:
        return "low"

In [19]:
df_feat['lifestyle_risk'] = df_feat.apply(lifestyle_risk,axis = 1)

In [20]:

tier_1_cities = ["Mumbai", "Delhi", "Bangalore", "Chennai", "Kolkata", "Hyderabad", "Pune"]
tier_2_cities = [
    "Jaipur", "Chandigarh", "Indore", "Lucknow", "Patna", "Ranchi", "Visakhapatnam", "Coimbatore",
    "Bhopal", "Nagpur", "Vadodara", "Surat", "Rajkot", "Jodhpur", "Raipur", "Amritsar", "Varanasi",
    "Agra", "Dehradun", "Mysore", "Jabalpur", "Guwahati", "Thiruvananthapuram", "Ludhiana", "Nashik",
    "Allahabad", "Udaipur", "Aurangabad", "Hubli", "Belgaum", "Salem", "Vijayawada", "Tiruchirappalli",
    "Bhavnagar", "Gwalior", "Dhanbad", "Bareilly", "Aligarh", "Gaya", "Kozhikode", "Warangal",
    "Kolhapur", "Bilaspur", "Jalandhar", "Noida", "Guntur", "Asansol", "Siliguri"
]
   

In [22]:
def city_tear(city):
    if city in tier_1_cities:
        return 1
    elif city in tier_2_cities:
        return 2
    else:
        return 3

In [23]:
df_feat["city_tear"] = df_feat["city"].apply(city_tear)

In [26]:
df_feat.drop(columns=['age', 'weight', 'height', 'smoker', 'city'])


,income_lpa,occupation,insurance_premium_category,bmi,age_group,lifestyle_risk,city_tear
0,2.92000,retired,High,49.227482,senior,low,2
1,34.28000,freelancer,Low,30.189017,adult,low,1
2,36.64000,freelancer,Low,21.118382,adult,low,2
3,3.34000,student,Medium,45.535900,young,high,1
4,3.94000,retired,High,24.296875,senior,low,2
...,...,...,...,...,...,...,...
95,19.64000,business_owner,Low,21.420747,adult,low,2
96,34.01000,private_job,Low,47.984483,adult,low,1
97,44.86000,freelancer,Low,18.765432,middle aged,low,1
98,28.30000,business_owner,Low,30.521676,adult,low,1


In [27]:
x = df_feat[["income_lpa","occupation",'bmi','age_group','lifestyle_risk','city_tear']]
y = df_feat['insurance_premium_category']

In [ ]:
categorical_feature = df_feat[["occupation", "age_group", "lifestyle_risk"]]
numerical_feature = df_feat[["bmi", "income_lpa"]]


In [33]:
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(), ["occupation", "age_group", "lifestyle_risk"]),
        ("num", "passthrough", ["bmi", "income_lpa"])
    ]
)

In [36]:

# Create a pipeline with preprocessing and random forest classifier
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [44]:


# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
pipeline.fit(X_train, y_train)

ValueError: No valid specification of the columns. Only a scalar, list or slice of all integers or all strings, or boolean mask is allowed